# Pyspark - Preprocessing

## Introduction

#### Description

Dans ce TP, vous allez approfondir vos connaissances sur PySpark, une bibliothèque Python très populaire pour l'analyse et la modélisation de données volumineuses. Ici, vous apprendrez à créer un ensemble de données à l'aide de la bibliothèque PySpark, et à le manipuler en utilisant des techniques de filtrage et de découpage standard.

Vos compétences en matière de gestion des données seront mises à l'épreuve et, à la fin de ce laboratoire, vous devriez avoir une compréhension approfondie de la façon dont PySpark fonctionne en pratique pour construire des pipelines d'analyse de données.



#### Objectifs d'apprentissage

À la fin de ce laboratoire, vous serez en mesure de :

- Créer une Session Spark, et stocker les données dans un Spark DataFrame ;
- Interroger des données avec PySpark en utilisant le SQL standard ;
- Créer une nouvelle colonne à l'intérieur du Spark DataFrame ;
- Effectuer un nettoyage standard des données - cohérence des types, filtrage, découpage en tranches ;
- Pivoter et manipuler un DataFrame Spark.

#### Public Cible

Ce laboratoire est destiné à :
 - Ceux qui sont intéressés à effectuer des analyses de données avec Python.
 - Toute personne impliquée dans la science des données et les pipelines d'ingénierie.


## Prérequis

#### Connaissances :
Vous devez posséder :
 - Une compréhension intermédiaire de Python.
 - Une connaissance de base de SQL.
 - Une connaissance de base des bibliothèques suivantes : Pandas.

#### Prérequis installations

- Installer apache Spark
- Java 8 car spark est ecrit en scala et qu'il lui faut donc une JVM (java virtual machine).

In [ ]:
!apt install openjdk-8-jdk-headless -qq
!pip install pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

### Initialiser une SparkSession

Introduisons un objet très important dans `pyspark` : le `SparkSession`. La Session Spark est un point d'entrée unifié d'une application spark à partir de Spark 2.0. Avant son introduction, le `SparkContext` était le point d'entrée de toute application spark, et nécessitait un `SparkConf`, qui avait toutes les configurations de cluster et les paramètres pour créer un objet Spark Context.

Nous importons donc du sous-module Spark SQL la classe `SparkSession`, et nous instancions une session en utilisant la méthode `getOrCreate`.

In [1]:
# from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# conf = SparkConf().set("spark.ui.port", "4050")
# sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

22/06/28 07:50:19 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.6 instead (on interface en0)
22/06/28 07:50:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/28 07:50:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark SQL fournit `spark.read().csv("nom_fichier")` pour lire un fichier ou un répertoire de fichiers au format CSV dans Spark DataFrame : utilisons cette méthode pour ingérer notre jeu de données. Nous spécifions `header=True` pour importer la première ligne du csv comme en-tête du dataframe Spark.

In [2]:
file_path = "https://raw.githubusercontent.com/ADataGuru/labs/lab/map-reduce/modules/batch/spark/tp/data/tips.csv"
import pandas as pd
pd_df = pd.read_csv(file_path)
tips = spark.createDataFrame(pd_df)
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77|   2|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26|   5|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43|   3|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [3]:
type(tips)

pyspark.sql.dataframe.DataFrame

Comme nous pouvons le voir, une simple inspection de l'objet `tips` confirme que nous avons affaire à un DataFrame Spark.

Cool ! Mais il nous manque un aspect crucial, qui peut être expliqué comme suit. Techniquement parlant, un Spark DataFrame est conceptuellement équivalent à une table dans une base de données relationnelle, et il est fréquent d'utiliser cette définition pour identifier un Spark DataFrame. En particulier, les tables Spark peuvent être de deux types. Temporaires ou permanentes. Ces deux types de tables sont présents dans une base de données. Si nous ne spécifions pas de base de données, Spark utilise la base de données `par défaut`. Nous pouvons voir la liste des bases de données disponibles avec `listDatabases`.




In [6]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/Users/loic.caminale/Workspace/formation/dataguru/labs/modules/batch/spark-101/spark-warehouse')]

Si nous inspectons les tables disponibles, nous voyons facilement qu'il n'y a pas de tables disponibles.

In [5]:
spark.catalog.listTables('default')

[]

Pour créer une table dans la database par `default`, nous allons utiliser la méthode `createOrReplaceTempView`.

In [7]:
tips.createOrReplaceTempView("tips") # Add tips data to the catalog

In [8]:
spark.catalog.listTables()

[Table(name='tips', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### Requêter la donnée avec Pyspark

L'avantage de la table Spark est que nous pouvons exécuter n'importe quelle requête avec le SQL standard. Ainsi, par exemple, supposons que nous souhaitons inspecter les 10 premières lignes de la table `tips`. En SQL standard, cela se traduit par la requête suivante :

In [10]:
QUERY_TIPS = "FROM tips SELECT * LIMIT 10"

Nous pouvons maintenant passer la requête au module `sql` comme suit :

In [11]:
tips10 = spark.sql(QUERY_TIPS)
tips10.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77|   2|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+



Il existe une interaction entre un DataFrame (ou table) Spark et un DataFrame Pandas : en effet, nous pouvons convertir une table Spark en un DataFrame Pandas en utilisant la fonction `toPandas`. Cela peut être utile, en particulier pour les data scientists qui souhaitent utiliser Pandas au cas où ils auraient besoin d'effectuer des manipulations plus avancées ou simplement d'intégrer une pipeline Python plus complexe.

In [12]:
tips10 = spark.sql(QUERY_TIPS)
tips10_df = tips10.toPandas()
tips10_df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.5,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


Essayons d'écrire une requête sql `group by` comme suit. Nous voulons compter le nombre de clients par jour et par sexe. Pour cela, vous devez écrire une requête qui effectue les opérations suivantes :
 - SELECTIONNER les colonnes `jour`, `sexe` et `COUNT(*)` ;
 - depuis la table `tips` ;
 - GROUPER PAR les colonnes `day` et `sex` ;
 - ORDER PAR la colonne `day`.


In [13]:
STUDENT_QUERY = "SELECT day, sex, COUNT(*) as N FROM tips GROUP BY day, sex ORDER BY day"

In [14]:
tips_counts = spark.sql(STUDENT_QUERY)
pd_counts = tips_counts.toPandas() # Convert the results to a pandas DataFrame
pd_counts.shape

(8, 3)

## Créer une nouvelle colonne avec PySpark

Le traitement des données avec PySpark est assez simple. Par exemple, supposons que nous souhaitons ajouter une nouvelle colonne à la table spark `tips`. Pour ce faire, nous utilisons la méthode `withColumn`, qui retourne un nouveau DataFrame en ajoutant une colonne ou en remplaçant la colonne existante qui a le même nom. L'expression de la colonne doit être une expression sur ce DataFrame.

Supposons, par exemple, que nous voulions calculer le pourcentage de pourboires par rapport à la facture totale. Nous avons besoin de deux colonnes du tableau d'origine : le `tip` et le `total_bill`. Donc, dans le `withColumn`, nous spécifions :
 - le nom des nouvelles colonnes sous forme de chaîne. Dans notre cas : `perc_tips` ;
 - l'expression de la colonne basée sur les colonnes des tables existantes. Dans notre cas : `(tips.tip/tips.total_bill)*100`
 

In [15]:
tips = spark.table("tips")
tips = tips.withColumn("perc_tips", (tips.tip/tips.total_bill)*100)
tips.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77|   2|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26|   5|Female|    No|Sun|Dinner|   4|14.180374361883

### Pivoter une table Spark

Nous pouvons même effectuer des transformations de données directement avec pyspark : par exemple, supposons que nous soyons intéressés à obtenir le nombre total de clients par leur sexe. Nous pouvons appeler directement dans l'onglet `tips` la méthode `groupBy`, en spécifiant la colonne sur laquelle nous souhaitons agréger - dans notre cas `sex`.

Pour agréger sur la colonne `sex`, nous utilisons la méthode `count()`, qui calcule le nombre total d'occurrences par sexe.

In [16]:
by_sex = tips.groupBy("sex")
by_sex.count().show()

+------+-----+
|   sex|count|
+------+-----+
|Female|   87|
|  Male|  157|
+------+-----+



Nous pouvons évidemment spécifier une méthode d'agrégation différente : par exemple, nous pourrions être intéressés par le calcul du pourcentage moyen de conseils par sexe.

In [17]:
by_gender = tips.groupBy("sex")

In [18]:
by_gender.avg("perc_tips").show()

+------+------------------+
|   sex|    avg(perc_tips)|
+------+------------------+
|Female|16.649073632892485|
|  Male|15.765054700429744|
+------+------------------+



### Convertion de types

Parfois, nous voulons être sûrs que les données sont stockées correctement dans notre table. Une bonne caractéristique de la méthode `withColumn` est qu'elle permet également de remplacer des colonnes existantes, ce qui signifie que nous pouvons même remplacer le type existant de la colonne. Mais comment pouvons-nous faire cela ?

Eh bien, c'est assez simple : en appelant la méthode `withColumn`, on spécifie le nom de la colonne, puis l'expression de la colonne en appliquant la méthode `cast` sur celle-ci. De cette façon, nous pouvons facilement attribuer un nouveau type à une colonne existante.

In [19]:
tips = tips.withColumn("total_bill",tips.total_bill.cast("double"))
tips = tips.withColumn("tip", tips.tip.cast("double"))
tips = tips.withColumn("size", tips.size.cast("integer"))

In [20]:
tips.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

Essayons d'effectuer une agrégation en utilisant la méthode `groupBy` comme suit. Nous voulons calculer le pourboire moyen par sexe et par fumeur. Pour ce faire, vous devez appliquer la méthode `groupBy` sur les `tips` en spécifiant, à l'intérieur de l'appel `groupBy`, les colonnes `"smoker"` et `"sex"`, séparées par une virgule. Assurez-vous de stocker cet objet dans la variable `by_smoker_sex_table`.

Ensuite, appliquez sur la table `by_smoker_sex_table` le `avg("tip")`, qui agrège la valeur du pourboire par fumeur et par sexe. Ici, l'agrégation est le pourboire moyen par rapport aux colonnes spécifiées. Veillez à appeler la méthode `show()` à la fin.

In [21]:
by_smoker_sex_table = tips.groupBy("smoker", "sex")
by_smoker_sex_table.avg("tip").show()

+------+------+------------------+
|smoker|   sex|          avg(tip)|
+------+------+------------------+
|    No|Female|2.7735185185185185|
|    No|  Male|  3.11340206185567|
|   Yes|  Male|3.0511666666666666|
|   Yes|Female| 2.931515151515151|
+------+------+------------------+



In [22]:
by_smoker_sex_df = by_smoker_sex_table.avg("tip").toPandas() # Convert the results to a pandas DataFrame
by_smoker_sex_df.iloc[0,1] # must be "Female"

'Female'

### Découpage d'une table avec `select'.

In case we wish to select just a few columns, we can use the `select` method, which allows to select the desired columns, and it returns a temporary view.

In [ ]:
# Select the correct columns
tips_000 = tips.select("total_bill", "tip", "size", "perc_tips")
tips_000.show()

### Filtrer une table avec `filter`

Nous avons deux façons distinctes d'effectuer un filtrage avec une table Spark :
 1. en utilisant une expression de chaîne dans la méthode `filter` ;
 2. en utilisant une condition booléenne dans la méthode `filter`.

En particulier, ici nous voulons effectuer une condition de filtrage simple, à savoir choisir tous les enregistrements avec une `total_bill` supérieure à 40 USD.

#### Filtrage avec une condition string

In [23]:
tips_01 = tips.filter("total_bill>40").toPandas()
tips_01

,total_bill,tip,sex,smoker,day,time,size,perc_tips
0,48.27,6.73,Male,No,Sat,Dinner,4,13.942407
1,40.17,4.73,Male,Yes,Fri,Dinner,4,11.774956
2,44.30,2.50,Female,Yes,Sat,Dinner,3,5.643341
3,41.19,5.00,Male,No,Thur,Lunch,5,12.138869
4,48.17,5.00,Male,No,Sun,Dinner,6,10.379905
5,50.81,10.00,Male,Yes,Sat,Dinner,3,19.681165
6,45.35,3.50,Male,Yes,Sun,Dinner,3,7.717751
7,40.55,3.00,Male,Yes,Sun,Dinner,2,7.398274
8,43.11,5.00,Female,Yes,Thur,Lunch,4,11.598237
9,48.33,9.00,Male,No,Sat,Dinner,4,18.621974


#### Filtrage avec une condition booléen

In [24]:
tips_02 = tips.filter(tips.total_bill>40).toPandas() # Filter tips with a boolean
tips_02

,total_bill,tip,sex,smoker,day,time,size,perc_tips
0,48.27,6.73,Male,No,Sat,Dinner,4,13.942407
1,40.17,4.73,Male,Yes,Fri,Dinner,4,11.774956
2,44.30,2.50,Female,Yes,Sat,Dinner,3,5.643341
3,41.19,5.00,Male,No,Thur,Lunch,5,12.138869
4,48.17,5.00,Male,No,Sun,Dinner,6,10.379905
5,50.81,10.00,Male,Yes,Sat,Dinner,3,19.681165
6,45.35,3.50,Male,Yes,Sun,Dinner,3,7.717751
7,40.55,3.00,Male,Yes,Sun,Dinner,2,7.398274
8,43.11,5.00,Female,Yes,Thur,Lunch,4,11.598237
9,48.33,9.00,Male,No,Sat,Dinner,4,18.621974


Nous pouvons même appliquer plusieurs filtrages en chaine :

In [25]:
filterA = tips.sex == "Female"
filterB = tips.day == "Sun"
tips_03 = tips.filter(filterA).filter(filterB)

In [26]:
tips_03.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883155|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|20.364126770060686|
|     10.33|1.67|Female|    No|Sun|Dinner|   3| 16.16650532429816|
|     16.97| 3.5|Female|    No|Sun|Dinner|   3|20.624631703005306|
|     10.29| 2.6|Female|    No|Sun|Dinner|   2| 25.26724975704568|
|     34.81| 5.2|Female|    No|Sun|Dinner|   4|14.938236139040505|
|     25.71| 4.0|Female|    No|Sun|Dinner|   3|15.558148580318942|
|     17.31| 3.5|Female|    No|Sun|Dinner|   2|20.219526285384173|
|     29.85|5.14|Female|    No|Sun|Dinner|   5|17.219430485762143|
|      25.0|3.75|Female|    No|Sun|Dinner|   4|              1

Une autre application intéressante de cette fonctionnalité est que nous pouvons supprimer les valeurs nulles avec une simple chaîne de caractères. Par exemple, si l'on souhaite supprimer toutes les références pour lesquelles la valeur `total_bill` est nulle, il suffit de passer à l'appel `filter` la chaîne `"total_bill is not NULL"`. C'est cool, n'est-ce pas ?

In [27]:
tips_counts = tips.filter("total_bill is not NULL")
tips_counts.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|         perc_tips|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

Essayons d'effectuer une agrégation en utilisant la méthode `groupBy` comme suit. Comme nous l'avons fait précédemment, nous voulons calculer le pourboire moyen pour un homme non-fumeur. Pour ce faire, nous pouvons filtrer les `tips` par sexe et fumeur, comme suit :
 - filter by `tips.sex=='Male'`
 - filtrer par `tips.smoker=='No'`

et ensuite vous `groupBy().avg('perc_tips')`. Veillez à appeler la méthode `show()` à la fin.

In [29]:
exercise_table = tips.filter(tips.sex=='Male').filter(tips.smoker=='No').groupBy().avg('perc_tips')
exercise_table_df = exercise_table.toPandas()
exercise_table_df

,avg(perc_tips)
0,16.066872


In [ ]:
# ====================================
# Validation Check
# DO NOT CHANGE THIS CELL
# ====================================
vcf_02 =  int(exercise_table_df.iloc[0,0])
with open('results/vcf_02.txt', 'w') as f:
    f.write("%s\n" % vcf_02)